In [1]:
!pip -q install langchain==0.3.27 langchain-community==0.3.31 langchain_google_genai==2.1.12 langchain-core==0.3.79 faiss-cpu==1.12.0 python-dotenv==1.1.1 pypdf==6.1.2 serpapi==0.1.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


In [2]:
!pip show langchain langchain-community langchain_google_genai langchain-core faiss-cpu python-dotenv pypdf serpapi

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
---
Name: langchain-community
Version: 0.3.31
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-google-genai
Version: 2.1.12
Summary: An integration package connecting Google's genai package and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: filetype, google-ai-generativelanguage, langchain-core, pydantic
Req

In [3]:
from dotenv import load_dotenv
import os
from langchain_community.utilities import SerpAPIWrapper
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
# from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage,ToolMessage
import json
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [5]:
from google.colab import userdata
import os
# Load Gemini API key from Colab secrets
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
SERPAPI_API_KEY=userdata.get('SERPAPI_API_KEY')

In [6]:
messages = []

# ✅ Load API key
# load_dotenv()
# SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY") # Removed
if not SERPAPI_API_KEY: # Changed to use the variable loaded from Colab secrets
    raise ValueError("❌ SERPAPI_API_KEY not found. Please check your .env file.")

In [9]:
# ✅ Load and process PDF
pdf_path = "/content/scholarship_info.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(pages)

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=GOOGLE_API_KEY)
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [10]:
# ✅ Set up LLM and RAG chain
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=GOOGLE_API_KEY
)
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

In [11]:
# ✅ Define tools using modern `@tool` decorator
@tool
def pdf_scholarship_search(query: str) -> str:
    """Answer scholarship-related questions using the loaded PDF."""
    return rag_chain.invoke(query)["result"]

# @tool
# def web_search(query: str) -> str:
#     """Search the web using SerpAPI for up-to-date information."""
#     search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
#     return search.run(query)
@tool
def web_search(query: str) -> str:
    """Search the web using SerpAPI for up-to-date information."""
    try:
        search = SerpAPIWrapper()
        return search.run(query)
    except Exception as e:
        return f"❌ Failed to search the web: {str(e)}"

In [12]:
# ✅ Bind tools to LLM
llm_with_tools = llm.bind_tools([pdf_scholarship_search, web_search])

In [13]:
# ✅ Interactive loop
print("\n🎓 Welcome to the Scholarship Assistant!")
print("Type your question below (or type 'exit' to quit):")

messages = []

while True:
    user_input = input("\n🧠 Your question: ").strip()

    if user_input.lower() in ["exit", "quit"]:
        print("👋 Exiting. Good luck with your scholarships!")
        break

    # 1. Add user input
    messages.append(HumanMessage(content=user_input))

    # 2. LLM decides what to do
    ai_message = llm_with_tools.invoke(messages)
    messages.append(ai_message)

    # 3. If tool call, run tool and append result
    if ai_message.tool_calls:
        for call in ai_message.tool_calls:
            tool_name = call["name"]
            tool_args = call["args"]

            if tool_name == "web_search":
                tool_output = web_search.invoke(tool_args)
            elif tool_name == "pdf_scholarship_search":
                tool_output = pdf_scholarship_search.invoke(tool_args)
            else:
                tool_output = f"Unknown tool: {tool_name}"

            # 4. Append tool result as a message
            messages.append(ToolMessage(tool_call_id=call["id"], content=tool_output))

        # 5. Ask LLM to give final answer using tool result
        final_response = llm_with_tools.invoke(messages)
        print("\n📌 Answer:")
        print(final_response.content)
        messages.append(final_response)

    else:
        # No tool needed — just print answer
        print("\n📌 Answer:")
        print(ai_message.content)


🎓 Welcome to the Scholarship Assistant!
Type your question below (or type 'exit' to quit):

🧠 Your question: For whom is this scholarship available?

📌 Answer:
This scholarship is available for students in India who are pursuing undergraduate degrees, have an annual family income below ₹6,00,000, and scored a minimum of 60% marks in their last qualifying exam.

🧠 Your question: How much is the scholarship?

📌 Answer:
The scholarship offers ₹10,000 per semester for tuition and a book allowance of ₹3,000 per year.

🧠 Your question: quit
👋 Exiting. Good luck with your scholarships!
